In [0]:
# Describe the history of the table
history_df = spark.sql('''DESCRIBE HISTORY chipgraham.alert_workflow.active_sale_price_alerts''')

# Get the most recent operation
latest_row = history_df.orderBy(history_df['version'].desc()).first()
latest_op = latest_row['operation']

if latest_op in ("WRITE", "UPDATE"):
	# Define actual max version
	max_version = spark.sql('''SELECT MAX(version) as a
	FROM (DESCRIBE HISTORY chipgraham.alert_workflow.active_sale_price_alerts)''')
	max_version_value = max_version.collect()[0][0]
	change_type = spark.sql(f'''select _change_type
	from table_changes('chipgraham.alert_workflow.active_sale_price_alerts', {max_version_value})''').collect()[0][0]
else:
	# Find the most recent row with WRITE or UPDATE
	write_update_df = history_df.filter(history_df['operation'].isin(["WRITE", "UPDATE"]))
	most_recent_write_update_row = write_update_df.orderBy(write_update_df['version'].desc()).limit(1).collect()[0]
	most_recent_write_update_version = most_recent_write_update_row['version']
	# Display the row as a DataFrame
	change_type = spark.sql(f'''select _change_type
	from table_changes('chipgraham.alert_workflow.active_sale_price_alerts', {most_recent_write_update_version})''').collect()[0][0]

In [0]:
dbutils.jobs.taskValues.set(key="change_type", value=change_type)